Imports

In [ ]:
import preamble

import random
import os
import time
from robot import Robot
from console_command_thread import ConsoleCommandThread
from phrase_trial_data import PhraseTrialData
from vocalizer import Vocalizer
from stoppable_thread import StoppableThread
from virtual_dynamics import SimpleVirtualDynamics
from timer import Timer

Bake corpus (which is a subset of the universe of phrases)

In [ ]:
universe = {
    'prefix': [
        'I need you to',
        'I am telling you to',
        'I think you should',
        'It would be nice if you',
        'You should',
        'I think you should',
        'Can you',
        'Could you',
        'Please',
        'You must',
    ],
    'adverb': [
        'slightly',
        'greatly',
        'smoothly',
        'sharply',
        'slowly',
        'quickly',
        'lightly',
        'significantly',
        'softly',
        'harshly',
        'gradually',
        'immediately',
    ],
    'action': {
        'without target': [
            'move',
            'shift',
            'adjust',
            'proceed',
            'ease',
            'head',
            'scoot',
            'advance',
            'go',
        ],
        'with target': [
            'bring it',
            'take it',
            'move it',
            'shift it',
            'adjust it',
            'proceed it',
            'ease it',
            'scoot it',
            'advance it',
            'move yourself',
            'shift yourself',
            'adjust yourself',
            'proceed yourself',
            'ease yourself',
            'head yourself',
            'scoot yourself',
            'advance yourself',
        ],
    },
    'direction': [
        'left',
        'right',
        'up',
        'down',
        'forward',
        'backward',
    ],
}

def choose_random_part(universe, phrase_part_type):
    o = universe[phrase_part_type]
    if isinstance(o, dict):
        return choose_random_part(o, random.choice(list(o.keys())))
    elif isinstance(o, list):
        return random.choice(o)

phrase_categories = [
    ('direction',),
    ('action', 'direction'),
    ('adverb', 'action', 'direction'),
    ('prefix', 'action', 'direction'),
    ('prefix', 'adverb', 'action', 'direction'),
]

NUMBER_OF_USERS = 12
NUMBER_OF_EACH_CATEGORY = 8

corpus = [universe['direction']]

for phrase_category in phrase_categories[1:]:
    phrases_of_category = []
    for _ in range(NUMBER_OF_EACH_CATEGORY * NUMBER_OF_USERS):
        phrase_parts = []
        for phrase_part_type in phrase_category:
            phrase_parts.append(choose_random_part(universe, phrase_part_type))
        phrases_of_category.append(' '.join(phrase_parts))
    corpus.append(phrases_of_category)

output_dir = f"corpus__{time.time()}"
os.makedirs(output_dir, exist_ok=True)

for i, phrases_of_category in enumerate(corpus):
    with open(os.path.join(output_dir, f"{'_'.join(phrase_categories[i])}.txt"), "w") as f:
        f.write("\n".join(phrases_of_category))

Experiment

In [ ]:
CUSTOM_CORPUS = None # ex dir name: f'corpus__{unique_timestamp}'

if CUSTOM_CORPUS is not None:
    pass

USER_ID = 0

user_phrases = []

for category in corpus[1:]:
    for i in range(NUMBER_OF_EACH_CATEGORY):
        user_phrases.append(category[(USER_ID*NUMBER_OF_EACH_CATEGORY)+i])

random.shuffle(user_phrases)
        
user_phrases = corpus[0] + user_phrases

stop_running = False

vocalizer = Vocalizer(0)

save_trial = False

with Robot(
    '169.254.9.43',
    translational_force_deadband=6.0,
    rotational_force_deadband=0.5
) as r, ConsoleCommandThread([' ', 'restart']) as c:
    AXES = Robot.TRANSLATION

    init_point = r.get_pose(AXES)

    timer = Timer()

    vd = SimpleVirtualDynamics(
        M=20,
        B=50,
        K=0,
    )

    i = 0

    while i in range(len(user_phrases)):
        # get phrase for this trial
        phrase = user_phrases[i]

        # initiate phrase trial data class
        phrase_trial_data = PhraseTrialData(phrase, USER_ID)

        # start phrase trial recording
        while True:
            if c.poll_command(' '):
                break
            if c.poll_command('exit'):
                stop_running = True
                break
        if stop_running:
            break

        vocalizer.utter(phrase, True)

        # phrase trial recording
        while True:
            time = timer.t()

            dt = timer.dt()

            position = r.get_pose(AXES)

            velocity = r.get_velocity(AXES)

            force = r.get_force(AXES)

            phrase_trial_data.append(
                time,
                dt,
                position,
                velocity,
                force,
            )

            vd.apply_force(force, dt)

            r.set_velocity(vd.get_velocity(), AXES)

            # phrase trial recording commands
            if c.poll_command(' '):
                save_trial = True
                break
            if c.poll_command('restart'):
                # set iterator back 1 so that next phrase trial is just this current phrase trial again
                i -= 1
                break
            if c.poll_command('exit'):
                stop_running = True
                break
        if stop_running:
            break

        if save_trial:
            phrase_trial_data.save()
            save_trial = False

        # proceed to next phrase trial
        i += 1

        # reset back to init position
        r.set_pose(init_point, AXES)